In [141]:
import LinearAlgebra
const la = LinearAlgebra


LinearAlgebra

# Projection Methods


Example

In [142]:
X = [1 2. 1;
     2 4. 1;
     3 -4 2;
     1 0. 1]

4×3 Matrix{Float64}:
 1.0   2.0  1.0
 2.0   4.0  1.0
 3.0  -4.0  2.0
 1.0   0.0  1.0

### Gram Schimdt

$X =[x_1, x_2, ..., x_r]\in \mathbb{R}^{n\times r}, n>=r$ matrix with independent column vectors. 

In [143]:
function gram_schmidt(X::Matrix{Float64})
    # input: X is a nxr matrix with columns linear independent
    n, r = size(X)
    R = zeros(Float64, (r, r))
    Q = zeros(Float64, (n, r))

    R[1, 1] = la.norm(X[:, 1])

    if R[1, 1] < eps()
        return nothing
    end
    Q[:, 1] = X[:, 1]/R[1, 1]
    
    for j = 2:r
        R[1:j-1, j] = transpose(Q[:, 1:j-1])*X[:, j]    # [<q1, xj>, ..., <q[j-1], xj>]T
        q = X[:, j] - Q[:, 1:j-1]*R[1:j-1, j]           # [q1*r[1,j], ..., q[j-1]*r[j-1,j]]T
        R[j, j] = la.norm(q)
        if R[j, j] < eps()
            break
        end
        Q[:, j] = q/R[j, j]
    end
    return Q, R
end


gram_schmidt (generic function with 1 method)

In [144]:
Q, R = gram_schmidt(X)
display(Q)
display(R)

4×3 Matrix{Float64}:
 0.258199   0.35688    0.658693
 0.516398   0.71376   -0.447912
 0.774597  -0.602235  -0.118565
 0.258199   0.022305   0.592824

3×3 Matrix{Float64}:
 3.87298  -0.516398   2.58199
 0.0       5.97774   -0.111525
 0.0       0.0        0.566476

In [145]:
function modified_gram_schmidt(X::Matrix{Float64})
    # input: X is a nxr matrix with columns linear independent
    n, r = size(X)
    R = zeros(Float64, (r, r))
    Q = zeros(Float64, (n, r))

    R[1, 1] = la.norm(X[:, 1])
    
    if R[1, 1] < eps()
        return nothing
    end
    Q[:, 1] = X[:, 1]/R[1, 1]

    for j = 2:r
        q = X[:, j]
        for i = 1:j-1
            R[i, j] = la.dot(q, Q[:, i])
            q = q - R[i, j]*Q[:, i]
        end
        R[j, j] = la.norm(q)
        if R[j, j] < eps()
            break
        end
        Q[:, j] = q/R[j, j]
    end
    return Q, R
end


modified_gram_schmidt (generic function with 1 method)

In [146]:
Q, R = modified_gram_schmidt(X)
display(Q)
display(R)

4×3 Matrix{Float64}:
 0.258199   0.35688    0.658693
 0.516398   0.71376   -0.447912
 0.774597  -0.602235  -0.118565
 0.258199   0.022305   0.592824

3×3 Matrix{Float64}:
 3.87298  -0.516398   2.58199
 0.0       5.97774   -0.111525
 0.0       0.0        0.566476

### Householder

In [147]:
function e_vector(n:: Int64, k:: Int64)
    e = zeros(n)
    e[k] = 1
    return e
end

function sign(num:: Real)
    return (num >= 0) ? 1 : -1
end

function identity(n:: Int64, m:: Int64)
    return Matrix{Float64}(la.I, n, m)
end

identity (generic function with 1 method)

In [148]:
function householder(X:: Matrix{Float64})
    n, m = size(X)
    R = zeros((n, m))
    Q = zeros((n, m))
    I = Matrix{Float64}(la.I, n, n)
    P_cum = I
    for k = 1:m
        if k > 1
           R[:, k] = P_cum' * X[:, k]
        end
        beta = sign(X[k, k]) * la.norm(X[k:end, k])
        z = zeros(n)
        for i = 1:n
            if i == k
                z[i] = beta + X[k, k]
            elseif i > k
                z[i] = X[i, k]
            end
        end
        w_k = z / la.norm(z)
        P = I - 2 * w_k * w_k'
        P_cum = P_cum * P
        R[:, k] = P * R[:, k]
        Q[:, k] = P_cum * e_vector(n, k)
    end
    return Q, R
end

householder (generic function with 1 method)

In [172]:
abs.([-15 4])

1×2 Matrix{Int64}:
 15  4

In [173]:
function modified_householder(X:: Matrix{Float64})
    n, m = size(X)
    R = copy(X)
    Q = identity(n, n)
    for k = 1:m
        if maximum(abs.(R[k:m, k])) == 0
            continue
        end
        beta = sign(R[k, k]) * la.norm(R[k:end, k])

        z = R[k:end, k] + beta * e_vector(n - k + 1, 1)        
        w  = z / la.norm(z)
        P = identity(n - k + 1, n - k + 1) - 2 * w * w'
        R[k:end, k:end] = P * R[k:end, k:end]
        Q[k:end, :] = P* Q[k:end, :]
    end
    return Q'[:, 1:end-1], R[1:end-1, :]
end

modified_householder (generic function with 1 method)

In [174]:
Q, R = modified_householder(X)
display(Q) 
display(R) 

4×3 Matrix{Float64}:
 -0.258199  -0.35688    0.658693
 -0.516398  -0.71376   -0.447912
 -0.774597   0.602235  -0.118565
 -0.258199  -0.022305   0.592824

3×3 Matrix{Float64}:
 -3.87298       0.516398     -2.58199
  4.16334e-17  -5.97774       0.111525
  4.16334e-16  -4.85723e-16   0.566476